## Introduction

The goal of this project is to determine the overall best neighborhood in Pittsburgh to live in. I chose public safety as my submetric, and in order to measure this, I will use crime statistics to determine the area with the least amount of crime.

## The Metric

For my calculations, I will analyze a police incident blotter dataset from the WPRDC. From this dataset I will keep track of each crime and where they took place. To the best of my ability, I will assign a value to each type of crime which will correspond to it's severity. This severity value will be a floating point number between 0 and 1. For each neighborhood, I will add together severity scores to determine a overall score based on the number and types of crimes that took place in each area. The area with the lowest score is the "safest" neighborhood while the highest score is the "least safe" neighborhood. To make comparisons easier, all other neighborhoods will get a score between 0 and 100 based on how close they score to the best neighborhood. (The best one would get a 100, whereas the worst would get a 0)

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

crime = pd.read_csv("crime.csv")
crime_types = ["Noise Complaint","Murder","Rape","Robbery","Aggravated Assault","Burglary","Theft","Vehicle Theft","Arson","Forgery","Harassment","Identity Theft","Theft Again","Receiving Stolen Property","Property Damage","Illegal Firearms","Prostitution","Indecent Exposure","Illegal Marijuana Possession/Sale","Illegal Gambling","Child Abuse/Endangerment","Drunk Driving","Public Consumption of Alcohol","Public Intoxication","Disorderly Conduct","Vagrancy","Stalking","Juvenile Runaways","Human Trafficking","Ignore this one"]
crime_values = {0:0.02,1:0.95,2:0.65,3:0.35,4:0.60,5:0.4,6:0.2,7:0.35,8:0.5,9:0.15,10:0.15,11:0.3,12:0.2,13:0.125,14:0.4,15:0.2,16:0.06,17:0.15,18:0.04,19:0.35,20:0.55,21:0.4,22:0.04,23:0.08,24:0.15,25:0.03,26:0.25,29:0.04,30:0.85,99:0.00}
crime_weights = {}
counter = 0

for val in crime_values.values():
    crime_weights[crime_types[counter]] = val
    counter += 1
crime_severity = pd.Series(crime_weights)
crime_severity.sort_values(ascending=False,inplace=True)
print("Please note that these values are completely subjective. Attempting to create this scale was not easy and it is certainly not perfect.")
print("Crime Type                           Severity Value")
display(crime_severity)

Please note that these values are completely subjective. Attempting to create this scale was not easy and it is certainly not perfect.
Crime Type                           Severity Value


Murder                               0.950
Human Trafficking                    0.850
Rape                                 0.650
Aggravated Assault                   0.600
Child Abuse/Endangerment             0.550
Arson                                0.500
Property Damage                      0.400
Burglary                             0.400
Drunk Driving                        0.400
Robbery                              0.350
Vehicle Theft                        0.350
Illegal Gambling                     0.350
Identity Theft                       0.300
Stalking                             0.250
Illegal Firearms                     0.200
Theft Again                          0.200
Theft                                0.200
Indecent Exposure                    0.150
Harassment                           0.150
Forgery                              0.150
Disorderly Conduct                   0.150
Receiving Stolen Property            0.125
Public Intoxication                  0.080
Prostitutio

In [39]:
area_scores = {}
for index, row in crime.iterrows():
    if row["INCIDENTNEIGHBORHOOD"] in area_scores:
        area_scores[row["INCIDENTNEIGHBORHOOD"]] += crime_values[row["HIERARCHY"]]
    else:
        area_scores[row["INCIDENTNEIGHBORHOOD"]] = crime_values[row["HIERARCHY"]]

## The Best Neighborhood

In [46]:
max_value = 0
for key, val in area_scores.items():
    if val > max_value:
        max_value = val
crime_scores = {}
for key, val in area_scores.items():
    crime_scores[key] = round(100 - val/max_value * 100, 1)
areas = pd.Series(area_scores.keys())
values = pd.Series(area_scores.values())
scores = pd.Series(crime_scores.values())
merged_data = pd.concat([areas,values,scores], axis=1,)
merged_data.rename({0:"Neighborhood", 1:"Crime Value", 2:"Crime Score"}, axis="columns",inplace=True)
merged_data.sort_values("Crime Value",ascending=True,inplace=True)
merged_data.head(99)

,Neighborhood,Crime Value,Crime Score
86,Outside County,14.945,99.5
94,Mt. Oliver Boro,15.280,99.5
92,Mt. Oliver Neighborhood,26.200,99.1
83,Outside State,28.285,99.1
81,Ridgemont,42.370,98.6
32,Troy Hill-Herrs Island,45.930,98.5
88,Chartiers City,45.945,98.5
93,New Homestead,49.085,98.4
16,East Carnegie,63.250,97.9
85,Swisshelm Park,69.355,97.7


In [41]:
cast = merged_data.astype(str)
plt.figure(figsize=(240,240), dpi=50)
worst_bar = plt.bar("Neighborhood","Crime Score", data=cast)